In [1]:
import pandas as pd
import numpy as np

import sklearn

In [2]:
from sklearn.metrics import make_scorer

def mean_absolute_percentage_error(y_true, y_pred): 
    return np.mean(np.abs((y_true - y_pred) / y_true))

mape = make_scorer(mean_absolute_percentage_error, greater_is_better=False)

In [3]:
data = pd.read_csv('../volume&weather_outlier.csv').drop(['Unnamed: 0'], axis = 1)

In [4]:
train_x = data.drop(['time_window_start', 'volume'],  axis = 1)
for col in train_x.columns:
    train_x[col][train_x[col].isnull()] = -1

train_x = train_x.as_matrix()

C:\Users\yecheng\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [5]:
train_y = data.volume.as_matrix()

In [6]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold

from scipy.stats import randint as sp_randint

from sklearn.model_selection import RandomizedSearchCV
clf = GradientBoostingRegressor(n_estimators= 100)

param = {
    'learning_rate': [0.1, 0.2],
    'n_estimators': sp_randint(20, 400),
    'max_depth': sp_randint(2,10),
}

clf = RandomizedSearchCV(
    estimator=clf,
    param_distributions=param, 
    scoring=mape, 
    n_iter=10,
    cv=5,
    n_jobs=1
)

clf.fit(train_x, train_y)

gkf = KFold(5)
i = 0
for train, test in gkf.split(train_x, train_y):
    clf.best_estimator_.fit(train_x[train], train_y[train])
    print('* {}: train:{}, test:{}'.format(i,
        mean_absolute_percentage_error(train_y[train], clf.best_estimator_.predict(train_x[train])),
        mean_absolute_percentage_error(train_y[test], clf.best_estimator_.predict(train_x[test]))))
    i+=1

* 0: train:0.08915276955559409, test:0.1890202583815016
* 1: train:0.08374699012553521, test:0.24546074726658978
* 2: train:0.08041450399700523, test:0.3117732336802884
* 3: train:0.0934829829789243, test:0.22058025176015345
* 4: train:0.09091841782926988, test:0.18101048511142984


In [8]:
clf.best_params_

{'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 103}

In [ ]:
test_data = pd.read_csv('../volume&weather_outlier.csv').drop(['Unnamed: 0'], axis = 1)
test_x = data.drop(['time_window_start', 'volume'],  axis = 1)

